# Assignment 3
All questions are weighted the same in this assignment. This assignment requires more individual learning then the last one did - you are encouraged to check out the [pandas documentation](http://pandas.pydata.org/pandas-docs/stable/) to find functions or methods you might not have used yet, or ask questions on [Stack Overflow](http://stackoverflow.com/) and tag them as pandas and python related. All questions are worth the same number of points except question 1 which is worth 23% of the assignment grade.

**Note**: Questions 2-12 rely on your question 1c answer.

In [ ]:
import re
import pandas as pd
import numpy as np

# Filter all warnings. If you would like to see the warnings, please comment the two lines below.
import warnings
warnings.filterwarnings('ignore')

### Question 1(a)

Complete the function `load_data` below to load three datasets that we will use in subsequent questions. Be sure to follow the instructions below for each dataset *respectively*. 



**Energy**

Load the energy data from the file `assets/Energy Indicators.xls`, which is a list of indicators of [energy supply and renewable electricity production](assets/Energy%20Indicators.xls) from the [United Nations](http://unstats.un.org/unsd/environment/excel_file_tables/2013/Energy%20Indicators.xls) for the year 2013, and should be put into a DataFrame with the variable name of `energy`.

Keep in mind that this is an Excel file, and not a comma separated values file. Also, make sure to exclude the footer and header information from the datafile. The first two columns are unneccessary, so you should get rid of them, and you should change the column labels so that the columns are:

`['Country', 'Energy Supply', 'Energy Supply per Capita', '% Renewable]`

Convert `Energy Supply` to gigajoules (**Note: there are 1,000,000 gigajoules in a petajoule**). For all countries which have missing data (e.g. data with "...") make sure this is reflected as `np.NaN` values.

Rename the following list of countries (for use in later questions):

```"Republic of Korea": "South Korea",
"United States of America": "United States",
"United Kingdom of Great Britain and Northern Ireland": "United Kingdom",
"China, Hong Kong Special Administrative Region": "Hong Kong"```

There are also several countries with parenthesis in their name. Be sure to remove these, e.g. `'Bolivia (Plurinational State of)'` should be `'Bolivia'`.



**GDP**

Next, load the GDP data from the file `assets/world_bank.csv`, which is a csv containing countries' GDP from 1960 to 2015 from [World Bank](http://data.worldbank.org/indicator/NY.GDP.MKTP.CD). Call this DataFrame `gdp`. 

Make sure to skip the header, and rename the following list of countries:

```"Korea, Rep.": "South Korea", 
"Iran, Islamic Rep.": "Iran",
"Hong Kong SAR, China": "Hong Kong"```



**ScimEn**

Finally, load the [Sciamgo Journal and Country Rank data for Energy Engineering and Power Technology](http://www.scimagojr.com/countryrank.php?category=2102) from the file `assets/scimagojr-3.xlsx`, which ranks countries based on their journal contributions in the aforementioned area. Call this DataFrame `scim_en`.

**For all three datasets, use country names as the index.**

In [ ]:
import re
word= "Bolivia (Plurinational State of)"
newword = re.sub(r' \([\w\s]*\)', r'', word)


In [ ]:
gdp = pd.read_csv("assets/world_bank.csv")
    
colnames_list = gdp.iloc[3].tolist()
colnames =[]

colnames = [x  if isinstance(x, str) else int(x) for x in colnames_list] 	
gdp.columns = colnames
gdp = gdp.iloc[4:, :]
gdp['Country Name']=gdp['Country Name'].replace({"Korea, Rep.": "South Korea", "Iran, Islamic Rep.": "Iran","Hong Kong SAR, China": "Hong Kong"})
gdp.set_index('Country Name', inplace = True) 


In [ ]:
scim_en =  pd.read_excel('assets/scimagojr-3.xlsx')
scim_en.head()

In [ ]:
def load_data():
    # Competency: reading files in Pandas, df manipulation, regex
    
    # The three variables are initialized to None. You will fill them with the correct values. 
    energy, gdp, scim_en = [None] * 3
    
    energy = pd.read_excel('assets/Energy Indicators.xls') 
    energy = energy.iloc[17:244, 2:]
    energy.columns = ['Country', 'Energy Supply', 'Energy Supply per Capita', '% Renewable']
    
    energy['Energy Supply'] = energy['Energy Supply'].replace(['...'], np.NaN)
    energy['Energy Supply'] = energy['Energy Supply'].apply(lambda x:x*1000000)
   
    energy['Country'] = energy['Country'].apply(lambda x:re.sub(r' \([\w\s]*\)', r'', x))
    energy['Country'] = energy['Country'].apply(lambda x:re.sub(r'\d+', r'', x))
    
    energy['Country'] = energy['Country'].replace({"Republic of Korea": "South Korea", "United States of America": "United States", "United Kingdom of Great Britain and Northern Ireland": "United Kingdom", "China, Hong Kong Special Administrative Region": "Hong Kong"})
    energy.set_index('Country', inplace = True) 
    gdp = pd.read_csv("assets/world_bank.csv")
    colnames_list = gdp.iloc[3].tolist()

    colnames = [x  if isinstance(x, str) else str(int(x)) for x in colnames_list] 	

    gdp.columns = colnames
    gdp = gdp.iloc[4:, :]

    gdp['Country Name']=gdp['Country Name'].replace({"Korea, Rep.": "South Korea", "Iran, Islamic Rep.": "Iran","Hong Kong SAR, China": "Hong Kong"})
    gdp.rename({'Country Name': 'Country'}, axis=1, inplace=True)
    gdp.set_index('Country', inplace = True) 

    scim_en =  pd.read_excel('assets/scimagojr-3.xlsx')
    scim_en.set_index('Country', inplace = True)
    return energy, gdp, scim_en



In [ ]:
# Cell for autograder.


### Question 1(b)

Now suppose we take the intersection of the three datasets based on the country names, how many *unique* entries will we lose? Complete the function below that returns the answer as a single number. The Venn diagram in the next cell is worth a thousand words. 

*This function should return a single (whole) number.*

In [ ]:
%%HTML
<svg width="800" height="300">
  <circle cx="150" cy="180" r="80" fill-opacity="0.2" stroke="black" stroke-width="2" fill="blue" />
  <circle cx="200" cy="100" r="80" fill-opacity="0.2" stroke="black" stroke-width="2" fill="red" />
  <circle cx="100" cy="100" r="80" fill-opacity="0.2" stroke="black" stroke-width="2" fill="green" />
  <line x1="150" y1="125" x2="300" y2="150" stroke="black" stroke-width="2" fill="black" stroke-dasharray="5,3"/>
  <text x="300" y="165" font-family="Verdana" font-size="35">Everything but this!</text>
</svg>

In [ ]:
def answer_1b():
    # Competency: joining datasets, sets
    # energy, gdp,scim_en
    energy, gdp, scim_en=  load_data()
    df1 = pd.merge(energy, gdp, on='Country')
    df2 = pd.merge(df1, scim_en, on = 'Country')
    
    df3 = pd.merge(energy, gdp, on='Country', how='outer')
    df4 = pd.merge(df3, scim_en, on = 'Country', how='outer')
    return (df4.shape[0]-df2.shape[0])


In [ ]:
# Cell for autograder.


### Question 1(c)

Join the three datasets to form a new dataset, using the intersection of country names. Keep only the last 10 years (2006-2015) of GDP data and only the top 15 countries by Scimagojr 'Rank' (Rank 1 through 15). 

The index of the resultant DataFrame should still be the name of the country, and the columns should be 

```['Rank', 'Documents', 'Citable documents', 'Citations', 'Self-citations',
    'Citations per document', 'H index', 'Energy Supply',
    'Energy Supply per Capita', '% Renewable', '2006', '2007', '2008',
    '2009', '2010', '2011', '2012', '2013', '2014', '2015']```.

*This function should return a DataFrame with 20 columns and 15 entries.*

In [ ]:
def answer_1c():
    # Competency: df manipulation, joining datasets
    energy, gdp, scim_en=  load_data()
    df = pd.merge(energy, gdp, on='Country')
    df = pd.merge(df, scim_en, on = 'Country')
    
    df = df[df['Rank'] < 16][['Rank', 'Documents', 'Citable documents', 'Citations', 'Self-citations',
    'Citations per document', 'H index', 'Energy Supply',
    'Energy Supply per Capita', '% Renewable','2006', '2007', '2008',
    '2009', '2010', '2011', '2012', '2013', '2014', '2015']]
    
    df.sort_values(by=['Rank'], axis=0, ascending=True, inplace=True)

    
    return df


In [ ]:
your_ans = answer_1c()

assert isinstance(your_ans, pd.DataFrame), "Q1c: Your function should return a DataFrame."

assert your_ans.shape == (15, 20), "Q1c: Your resultant DataFrame should have 20 columns and 15 entries."

assert list(your_ans.columns) == ['Rank', 'Documents', 'Citable documents', 'Citations', 'Self-citations',
                                 'Citations per document', 'H index', 'Energy Supply','Energy Supply per Capita', '% Renewable', 
                                 '2006', '2007', '2008','2009', '2010', '2011', '2012', '2013', '2014', '2015'] , "Q1c: The column names should be as specified in the question. "
del your_ans

In [ ]:
# Cell for autograder.


**Note: all subsequent questions rely on the DataFrame returned by your function in Question 1(c) above.**

### Question 2
What is the average GDP over the last 10 years for each country?

*This function should return a Series named `avgGDP` with 15 countries and their average GDP sorted in descending order.*

In [ ]:
def answer_two():
    # Competency: indexing, math fn, sorting
    df=answer_1c()
    df = df[['2006', '2007', '2008', '2009', '2010', '2011', '2012', '2013', '2014', '2015']] 
    df=df.stack()
    result = df.groupby('Country').apply(lambda x: np.mean(x))
    result = result.rename('avgGDP')
    result.sort_values(ascending=False, inplace=True)
    return result


In [ ]:
your_ans = answer_two()

assert isinstance(your_ans, pd.Series), "Q2: You should return a Series. "
assert your_ans.name == "avgGDP", "Q2: Your Series should have the correct name. "

del your_ans

In [ ]:
# Cell for autograder.


### Question 3

By how much had the GDP changed over the 10 year span for the country with the 6th largest average GDP?

*This function should return a single number.*

In [ ]:
def answer_three():
    # Competency: indexing, broadcasting
    top_gdp=answer_two()
    sixth = top_gdp.index[5]
    df=answer_1c()
    gdp_sixth = df[df.index == sixth]
    change = gdp_sixth['2015'] - gdp_sixth['2006']
    return change.values[0]
 

In [ ]:
# Cell for autograder.


### Question 4

What is the mean energy supply per capita?

*This function should return a single number.*

In [ ]:
def answer_four():
    # Competency: math fn
    df=answer_1c()
    return df['Energy Supply per Capita'].mean()


In [ ]:
# Cell for autograder.


### Question 5

What country has the maximum % Renewable and what is the percentage?

*This function should return a tuple with the name of the country and the percentage.*

In [ ]:
def answer_five():
    # Competency: math fn
    df=answer_1c()
    max_renewable = df['% Renewable'].max()
    country = df[df['% Renewable'] == max_renewable].index[0]
    return (country, max_renewable)


In [ ]:
your_ans = answer_five()

assert isinstance(your_ans, tuple), "Q5: Your function should return a tuple. "

assert isinstance(your_ans[0], str), "Q5: The first element in your result should be the name of the country. "

del your_ans

In [ ]:
# Cell for autograder.


### Question 6

Create a new column that is the ratio of `Self-Citations` to total `Citations`. 
What is the maximum value for this new column, and what country has the highest ratio?

*This function should return a tuple with the name of the country and the ratio.*

In [ ]:
def answer_six():
    # Competency: math fn, broadcasting
    df=answer_1c()
    df['citation-ratio'] = df['Self-citations']/df['Citations']
    max_ratio =  df['citation-ratio'].max()
    country = df[df['citation-ratio'] == max_ratio].index[0]
    return (country, max_ratio)


In [ ]:
your_ans = answer_six()

assert isinstance(your_ans, tuple), "Q6: Your function should return a tuple. "

assert isinstance(your_ans[0], str), "Q6: The first element in your result should be the name of the country. "

del your_ans

In [ ]:
# Cell for autograder.


### Question 7

Create a column that estimates the population using `Energy Supply` and `Energy Supply per capita`. 
What is the third most populous country according to this estimate?

*This function should return the name of the country*

In [ ]:
def answer_seven():
    # Competency: Broadcasting, sorting
    df=answer_1c()
    df['estimated-population'] = df['Energy Supply']/df['Energy Supply per Capita']
    df.sort_values(by='estimated-population', ascending=False, inplace=True)
    
    country = df.index[2]
    return country


In [ ]:
assert isinstance(answer_seven(), str), "Q7: Your function should return the name of the country. "

In [ ]:
# Cell for autograder.


### Question 8

Create a column that estimates the number of citable documents per person. 
What is the correlation between the number of citable documents per capita and the energy supply per capita? Use the `.corr()` method, (Pearson's correlation).

*This function should return a single number.*

In [ ]:
import scipy.stats as stats

def answer_eight():
    # Competency: Broadcasting, math fn, correlation, visualization
    df=answer_1c()
    df['citable-doc-per-person'] = df['Citable documents']/(df['Energy Supply']/df['Energy Supply per Capita'])
 
    corr, pval=stats.pearsonr(df["citable-doc-per-person"],df["Energy Supply per Capita"])
    return corr
   

In [ ]:
assert -1 <= answer_eight() <= 1, "Q8: A valid correlation should be between -1 to 1. "

In [ ]:
# Cell for autograder.


### Question 9

Create a new column with a 1 if a country's `% Renewable` value is **at or above** the median, and a 0 otherwise for all countries in the top 15. 

*This function should return a series named `HighRenew` whose index is the country name sorted in ascending order of rank.*

In [ ]:
def answer_nine():
    # Competency: df querying, math fn, variable encoding
    df=answer_1c()
    renewable_median =df['% Renewable'].median()
    
    df['HighRenew'] = df['% Renewable'].apply(lambda x: 1 if x >= renewable_median else 0)
    df.sort_values(by='Rank', ascending=True, inplace=True)
    return df['HighRenew']


In [ ]:
assert isinstance(answer_nine(), pd.Series), "Q9: Your function should return a Series. "

In [ ]:
# Cell for autograder.


### Question 10

Use the following dictionary to group the `Countries` by `Continent`, then create a DataFrame that displays the sample size (the number of countries in each continent bin), and the sum, mean, and *population* standard deviation of the estimated population for each country. 

```python
ContinentDict  = {'China':'Asia', 
                  'United States':'North America', 
                  'Japan':'Asia', 
                  'United Kingdom':'Europe', 
                  'Russian Federation':'Europe', 
                  'Canada':'North America', 
                  'Germany':'Europe', 
                  'India':'Asia',
                  'France':'Europe', 
                  'South Korea':'Asia', 
                  'Italy':'Europe', 
                  'Spain':'Europe', 
                  'Iran':'Asia',
                  'Australia':'Australia', 
                  'Brazil':'South America'}
```

*This function should return a DataFrame with index named Continent `['Asia', 'Australia', 'Europe', 'North America', 'South America']` and columns `['size', 'sum', 'mean', 'std']`*

**Special Note: make sure you are indeed calculating the *population* std rather than the sample std. When in doubt, carefully check the documentation of the function you plan to use. Don't take things for granted.**

In [ ]:
def answer_ten():
    # Competency: mapping, groupby, agg
    
    ContinentDict  = {'China':'Asia', 
                  'United States':'North America', 
                  'Japan':'Asia', 
                  'United Kingdom':'Europe', 
                  'Russian Federation':'Europe', 
                  'Canada':'North America', 
                  'Germany':'Europe', 
                  'India':'Asia',
                  'France':'Europe', 
                  'South Korea':'Asia', 
                  'Italy':'Europe', 
                  'Spain':'Europe', 
                  'Iran':'Asia',
                  'Australia':'Australia', 
                  'Brazil':'South America'}
    
    df=answer_1c()
    df['estimated-population'] = df['Energy Supply']/df['Energy Supply per Capita']
    df['Country'] = df.index
    df['continent'] =  df['Country'].apply(lambda x: ContinentDict[x])
    
    df['estimated-population'] = df['estimated-population'].astype(float)
    
    result = df.groupby('continent').agg({'estimated-population': ['size', 'sum', 'mean', 'std']})
    result.columns = ['size', 'sum', 'mean', 'std']
    return result
                                      


In [ ]:
your_ans = answer_ten()

assert isinstance(your_ans, pd.DataFrame), "Q10: Your function should return a DataFrame. "

assert your_ans.shape[0] == 5, "Q10: You have a incorrect number of rows. "

assert your_ans.shape[1] == 4, "Q10: You have a incorrect number of columns. "

assert list(your_ans.index) == ['Asia', 'Australia', 'Europe', 'North America', 'South America'], "Q10: You have a wrong index. "

assert list(your_ans.columns) == ['size', 'sum', 'mean', 'std'], "Q10: You have wrong column names. "

assert np.isclose(your_ans.loc["Asia", "sum"], 2898666386.6106005, rtol=0.0, atol=1e-5), "Q10: The sum value for Asia should be around 2898666386.6106005. "

assert np.isclose(your_ans.loc["Europe", "mean"], 76321611.20272864, rtol=0.0, atol=1e-5), "Q10: The mean value for Europe should be around 76321611.20272864. "

assert np.isnan(your_ans.loc["South America", "std"]), "Q10: South America should have a NaN std. "

del your_ans

In [ ]:
# Cell for autograder.


### Question 11
Cut `% Renewable` into 5 bins. Group the top 15 countries by `Continent` as well as these new `% Renewable` bins. How many countries are there in each of these groups?

*This function should return a Series with a MultiIndex of `Continent`, then the bins for `% Renewable`. Do not include groups with no countries.*

In [ ]:
def answer_eleven():
    # Competency: cut, groupby, math fn
    ContinentDict  = {'China':'Asia', 
                  'United States':'North America', 
                  'Japan':'Asia', 
                  'United Kingdom':'Europe', 
                  'Russian Federation':'Europe', 
                  'Canada':'North America', 
                  'Germany':'Europe', 
                  'India':'Asia',
                  'France':'Europe', 
                  'South Korea':'Asia', 
                  'Italy':'Europe', 
                  'Spain':'Europe', 
                  'Iran':'Asia',
                  'Australia':'Australia', 
                  'Brazil':'South America'}
    
    df=answer_1c()
    df['Country'] = df.index
    df['Continent'] =  df['Country'].apply(lambda x: ContinentDict[x])
    
    df['% Renewable'] = pd.cut(df['% Renewable'], 5)
    result = df.groupby(['Continent','% Renewable'])['Country'].size()

    return result


In [ ]:
your_ans = answer_eleven()

assert isinstance(your_ans, pd.Series), "Q11: Your function should return a Series. "

assert len(your_ans) == 9, "Q11: Your answer should have 9 rows. "

del your_ans

In [ ]:
# Cell for autograder.


### Question 12

Write a function to display the estimated population as a string with thousands separator (using commas). Use all significant digits, namely, do not round the results.

e.g. 12345678.90 -> 12,345,678.90

*This function should return a series `PopEst` whose index is the country name and whose values are the population estimate string*

**Special Note: make sure your `Energy Supply` column is of a numerical type rather than "object".**

In [ ]:
def answer_twelve():
    # Competency: lambdas, format
    df=answer_1c()
    
    df['PopEst'] = df['Energy Supply']/df['Energy Supply per Capita']
    df['PopEst'] = df['PopEst'].apply(lambda x: str(format(x, ",")) )

    return df['PopEst']


In [ ]:
your_ans = answer_twelve()

assert isinstance(your_ans, pd.Series), "Q12: Your function should return a Series. "

assert len(your_ans) == 15, "Q12: There should be 15 countries. "

del your_ans

In [ ]:
# Cell for autograder.


### Optional

Use the built in function `plot_optional()` to see an example visualization.

In [ ]:
def plot_optional():
    import matplotlib as plt
    %matplotlib inline
    Top15 = answer_1c()
    ax = Top15.plot(x='Rank', y='% Renewable', kind='scatter', 
                    c=['#e41a1c','#377eb8','#e41a1c','#4daf4a','#4daf4a','#377eb8','#4daf4a','#e41a1c',
                       '#4daf4a','#e41a1c','#4daf4a','#4daf4a','#e41a1c','#dede00','#ff7f00'], 
                    xticks=range(1,16), s=6*Top15['2014']/10**10, alpha=.75, figsize=[16,6]);

    for i, txt in enumerate(Top15.index):
        ax.annotate(txt, [Top15['Rank'][i], Top15['% Renewable'][i]], ha='center')

    print("This is an example of a visualization that can be created to help understand the data. \
This is a bubble chart showing % Renewable vs. Rank. The size of the bubble corresponds to the countries' \
2014 GDP, and the color corresponds to the continent.")
    
plot_optional()